# Project: **A Classification Model of Restaurant Reviews through Natural Language Processing**
_____

Author: **Daniel Eduardo López**

#####<a href="https://github.com/DanielEduardoLopez"> GitHub </a>

#####<a href="https://www.linkedin.com/in/daniel-eduardo-lopez"> LinkedIn </a> 

### **1. Goal**

To develop several bag-of-words models for classificating restaurant reviews into positive or negative through Natural Language Processing (NLP) and selecting the one with the highest accuracy rate.
___
### **2. Research Question**

Which bag-of-words model for classificating restaurant reviews into positive or negative is able to yield the highest accuracy?
___
### **3. Hypothesis**

A bag-of-words model based on **Random Forests** for classificating restaurant reviews into positive or negative will yield the highest accuracy.
___
### **4. Methodology**

The methodology of the present study is based on Rollin’s Foundational Methodology for Data Science (Rollins, 2015).

1. **Analytical approach**: Building and evaluation of classification models.
2. **Data requirements**: Reviews of a restaurant and their corresponding labels (0 for negative and 1 for positive).
3. **Data collection**: Data was retrieved from <a href="https://www.kaggle.com/datasets/vigneshwarsofficial/reviews">Kaggle</a>.
4. **Data exploration and preparation**: Data then was explored and cleaned with Python 3 and its libraries Numpy, Pandas, Regular Expressions, and the Natural Language Toolkit.
5. **Data modeling**: 
6. **Evaluation**: 
___

### **5. Results**

#### **5.1 Data collection**
As mentioned before, data about restaurant reviews and its corresponding labels was retireved from <a href="https://www.kaggle.com/datasets/vigneshwarsofficial/reviews">Kaggle</a>.

In [ ]:
# Libraries importation
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

### **6. Conclusions**

### **7. References**
* **Rollins, J. B. (2015)**. *Metodología Fundamental para la Ciencia de Datos. Somers: IBM Corporation.* Retrieved from https://www.ibm.com/downloads/cas/WKK9DX51

In [ ]:
# End